# <center> Explonatory Shop Analysis

In [1]:
%store -r item_cat
%store -r item
%store -r sub
%store -r shops
%store -r sales_test
%store -r sales_train
%store -r __ipy
%store -r __da

In [2]:
import sys
import os
sys.path.append(os.path.realpath('./../../scripts'))
sys.path = list(set(sys.path))

In [3]:
%%capture
__ipy
__da

import plotly.express as px
from functools import partial
from scripts.basic_text_preprocessing import BasicPreprocessText

ModuleNotFoundError: No module named 'scripts.basic_text_preprocessing'

In [4]:
new_shops = shops.copy()
cleaned_shop_name = BasicPreprocessText().vectorize_process_text(shops['shop_name'])
new_shops['shop_name'] = cleaned_shop_name
new_shops['city'] = new_shops['shop_name'].apply(lambda x: x.split()[0])
city = new_shops['city'] .value_counts()\
.to_frame().reset_index().rename(columns={'index': 'shop_name', 'city': 'count_shops'})
px.bar(city[city['count_shops'] > 1], x='shop_name', y='count_shops')

NameError: name 'BasicPreprocessText' is not defined

Only one store present in city

In [53]:
city[city['count_shops'] == 1].count()

shop_name      18
count_shops    18
dtype: int64

In [147]:
def not_city_str(x, t):
    return 1 if t in "".join(x.split()[1:]) else 0
new_shops['is_mal'] = new_shops['shop_name'].apply(partial(not_city_str, t='тц'))
new_shops['is_en_mal'] = new_shops['shop_name'].apply(partial(not_city_str, t='трк'))

In [200]:
def get_location(x):
    loc = gmaps.geocode(x)
    return loc[0]['geometry']['location'] if len(loc) != 0 else {'lat': 0, 'lng': 0}

locations = new_shops['shop_name'].progress_apply(get_location) 





  0%|          | 0/60 [00:00<?, ?it/s]



  5%|▌         | 3/60 [00:00<00:03, 18.81it/s]



  7%|▋         | 4/60 [00:00<00:09,  5.80it/s]



 10%|█         | 6/60 [00:00<00:07,  7.11it/s]



 13%|█▎        | 8/60 [00:00<00:06,  8.25it/s]



 17%|█▋        | 10/60 [00:01<00:05,  9.66it/s]



 20%|██        | 12/60 [00:01<00:06,  7.84it/s]



 23%|██▎       | 14/60 [00:01<00:06,  7.07it/s]



 27%|██▋       | 16/60 [00:01<00:05,  8.26it/s]



 30%|███       | 18/60 [00:02<00:04,  9.52it/s]



 33%|███▎      | 20/60 [00:02<00:03, 10.62it/s]



 37%|███▋      | 22/60 [00:02<00:03, 11.29it/s]



 40%|████      | 24/60 [00:02<00:02, 12.15it/s]



 43%|████▎     | 26/60 [00:02<00:02, 12.99it/s]



 47%|████▋     | 28/60 [00:02<00:02, 13.77it/s]



 50%|█████     | 30/60 [00:02<00:02, 14.15it/s]



 53%|█████▎    | 32/60 [00:03<00:02, 12.77it/s]



 57%|█████▋    | 34/60 [00:03<00:02,  9.86it/s]



 60%|██████    | 36/60 [00:03<00:02, 10.89it/s]



 63%|██████▎   | 38/60 [00:03<00:01, 11.

In [205]:
new_shops_with_coords = pd.concat([new_shops, pd.DataFrame.from_records(locations.values)], axis=1)

In [206]:
new_shops_with_coords.head()

,shop_name,shop_id,city,is_mal,is_en_mal,lat,lng
0,якутск орджоникидзе 56 фран,0,якутск,0,0,62.03,129.72
1,якутск тц центральный фран,1,якутск,1,0,62.03,129.74
2,адыгея тц мега,2,адыгея,1,0,45.01,38.93
3,балашиха трк октябрькиномир,3,балашиха,0,1,55.80,37.94
4,волжский тц волга молл,4,волжский,1,0,48.77,44.80


In [221]:
plot_data = new_shops_with_coords[new_shops_with_coords['lng'] > 0]

data = [ go.Scattergeo(
        lon = plot_data['lng'][new_shops_with_coords['lng'] > 0],
        lat = plot_data['lat'],
        text = "Shop location")]

layout = dict(
        width=1000,
        height=500,
        title = 'Locations'
    )

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

Shop count: 60

In [56]:
print(f"Shop count: {sales_train['shop_id'].nunique()}")

Shop count: 60


# <center> Explonatory item Analysis

Duplication of item_id. Duplication of item_id's are absent

In [57]:
item['item_id'].shape[0] - item['item_id'].unique().shape[0]

0

Preprocess names to find simmilar names of items:

In [59]:
cleaned_item_name = BasicPreprocessText().vectorize_process_text(item['item_name'])
cleaned_item_name_series = pd.Series(cleaned_item_name)

NameError: name 'pd' is not defined

Duplication in item_name

In [ ]:
item['item_id'].shape[0] - cleaned_item_name_series.unique().shape[0]

Item count:

In [ ]:
print(f"Item count: {sales_train['item_id'].nunique()}")